In [1]:
import ollama
import pickle
def load_clients(pickle_path):
    try:
        with open(pickle_path, "rb") as f:
            clients = pickle.load(f)
        return clients
    except Exception as e:
        print(f"Failed to load pickle file: {e}")
        return None

full_data = load_clients("remaining_clients.pkl")

In [2]:
client_labels = []
for clients in full_data:
    if clients['label']['label'] == 'Reject':
        client_labels.append(False)
    elif clients['label']['label'] == 'Accept':
        client_labels.append(True)

In [ ]:
full_data[0]["client_profile"]

'single'

In [5]:
full_data[0]['client_description']['Family Background']

'Freja Katrine Christensen is currently single. She does not have any children.'

In [3]:
def check_family_background(client,model):
    input_text = 'marital status:'+ str(client["client_profile"]["marital_status"])+ ". "+ "Family Background:" + str(client["client_description"]["Family Background"])
    response = ollama.chat(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a logical AI assistant. Based on the user's input, check if the marital status "
                    "is coherent with the family background. Only reply with 'True' if they are coherent, otherwise reply with 'False'."
                )
            },
            {"role": "user", "content": input_text}
        ],
        options={"num_predict": 5, "temperature": 0.0})
    return response['message']['content']

In [59]:
full_data[4]['client_description']['Education Background']
full_data[1]["client_profile"]['secondary_school']
full_data[36]["client_profile"]['higher_education']


[{'university': 'Nyenrode Business Universiteit', 'graduation_year': 2022}]

In [117]:
for i in full_data:
    # Check if 'secondary_school' is of dictionary type
    if isinstance(i["client_profile"]['real_estate_details'], list):
        print(i["client_profile"]['real_estate_details'])
        continue
    else:
        print("It is NOT a dictionary:", str(type(i["client_profile"]['secondary_school'])))

[]
[]
[]
[{'property type': 'condo', 'property value': 995000, 'property location': 'Livorno'}, {'property type': 'villa', 'property value': 1830000, 'property location': 'Udine'}]
[{'property type': 'house', 'property value': 1790000, 'property location': 'Freiburg im Breisgau'}]
[{'property type': 'condo', 'property value': 1050000, 'property location': 'Reims'}, {'property type': 'house', 'property value': 2150000, 'property location': 'Rennes'}]
[{'property type': 'townhouse', 'property value': 820000, 'property location': 'Kuopio'}]
[{'property type': 'villa', 'property value': 2220000, 'property location': 'Freiburg im Breisgau'}, {'property type': 'villa', 'property value': 2510000, 'property location': 'Chemnitz'}, {'property type': 'condo', 'property value': 1210000, 'property location': 'Hamburg'}]
[{'property type': 'house', 'property value': 1330000, 'property location': 'Stockerau'}, {'property type': 'house', 'property value': 1229000, 'property location': 'Bregenz'}, {'p

In [60]:
def check_education_background(client,model):
    # Initialize an empty list to store the names of the educational institutions
    list_of_education = []
    if client["client_profile"]['secondary_school']['name']:
        list_of_education.append(client["client_profile"]['secondary_school']['name'])

    for higher in client["client_profile"]['higher_education']:
        list_of_education.append(higher['university'])

    # Combine the names of institutions into a sentence
    education_str = ', '.join(list_of_education)  # Joining names with commas for better readability

    # Construct the input text with the list of schools and the education background description
    input_text = 'Graduated from ' + education_str + ". " + "Education Background: " + str(client["client_description"]["Education Background"])
    response = ollama.chat(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a logical AI assistant. Based on the user's input, check if the education background matches "
                    "the listed schools. Only reply with 'True' if they match, otherwise reply with 'False'."
                )
            },
            {"role": "user", "content": input_text}
        ],
        options={"num_predict": 5, "temperature": 0.0}
    )
    return response['message']['content']

In [68]:
check_education_background(full_data[1],'gemma3')

'False\n'

In [76]:
def convert_to_natural_text(work_experience):
    # Loop through each work experience entry
    natural_text = []
    for entry in work_experience:
        start_year = entry['start_year']
        end_year = entry['end_year']
        company = entry['company']
        position = entry['position']
        
        # Check if end_year is None or a valid year
        if end_year is None:
            end_year_text = "currently working"
            text = f"from {start_year}, the individual currently working as an {position} at {company}"
        else:
            end_year_text = f"until {end_year}"
            text = f"from {start_year}, the individual worked as an {position} at {company} {end_year_text}"
        natural_text.append(text)
    
    return natural_text

In [ ]:
full_data[1]["client_profile"]['higher_education']

{'name': 'Staatliches Gymnasium am Obersalzberg', 'graduation_year': 2016}

In [74]:
full_data[1]["client_description"]["Occupation History"]

"Schmitt Scholz Schwarz is a 27 year old Art Dealer from Germany.\nSchmitt Scholz Schwarz's first role was as a Art Dealer at Bayerische Staatsgemäldesammlungen, where he worked diligently from 2022 till now to establish a strong reputation.\n"

In [113]:
full_data[1]["client_profile"]["aum"]

{'savings': 20000, 'inheritance': 2420000, 'real_estate_value': 0}

In [77]:
def check_work_background(client, model):
    # Convert structured employment data to natural language
    employment_history = convert_to_natural_text(client["client_profile"]["employment_history"])

    # Combine all work experiences into one readable paragraph
    employment_str = ' and '.join(employment_history)

    # Construct the input text for the model
    input_text = (
        "Employment history: " + employment_str + ". " +
        "Work History Description: " + str(client["client_description"]["Occupation History"])
    )

    # Chat with the model
    response = ollama.chat(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a logical AI assistant. Based on the user's input, check if the structured employment history "
                    "is coherent with the work background description. Only reply with 'True' if they are consistent, otherwise reply with 'False'."
                )
            },
            {"role": "user", "content": input_text}
        ],
        options={"num_predict": 5, "temperature": 0.0}
    )

    return response['message']['content']

In [83]:
check_work_background(full_data[79],'gemma3')

'True'

In [106]:
full_data[2]['client_profile']['currency']


'EUR'

In [ ]:
full_data[3]['client_profile']['inheritance_details'] #{'relationship': 'grandfather','inheritance year': 2016,'profession': 'Investment Banker'}
full_data[3]['client_profile']['real_estate_details'] #[{'property type': 'condo','property value': 995000,'property location': 'Livorno'},{'property type': 'villa','property value': 1830000,'property location': 'Udine'}]
full_data[3]['client_profile']['aum'] #{'savings': 20000, 'inheritance': 1350000, 'real_estate_value': 10000}
full_data[3]['client_profile']['currency'] # EUR

{'relationship': 'grandfather',
 'inheritance year': 2016,
 'profession': 'Investment Banker'}

In [119]:
full_data[3]['client_profile']['real_estate_details']

[{'property type': 'condo',
  'property value': 995000,
  'property location': 'Livorno'},
 {'property type': 'villa',
  'property value': 1830000,
  'property location': 'Udine'}]

In [120]:
full_data[3]['client_profile']['aum']

{'savings': 1450000, 'inheritance': 2690000, 'real_estate_value': 2825000}

In [108]:
full_data[2]['client_profile']['preferred_markets']

['Italy', 'Netherlands']

In [121]:
full_data[3]['client_description']['Wealth Summary']

'While working, she saved 1450000 EUR, which she used to build a diversified investment portfolio.\nShe has condo located in Livorno, worth 995000 EUR.\n\nShe purchased villa in Udine, where she spends her holidays. It is valued at 1830000 EUR.\n\nIn addition to her career success, she also received an inheritance of 2690000 EUR from her late mother, a renowned Corporate Lawyer, in 2013.\n'

In [123]:
def generate_wealth_summary_text(client_profile):
    # Currency symbol or code
    currency = client_profile['client_profile']['currency']
    aum = client_profile['client_profile']['aum']
    savings = aum['savings']
    inheritance_value = aum['inheritance']
    real_estate_value = aum['real_estate_value']
    inheritance = client_profile['client_profile']['inheritance_details']

    inheritance_text = (
        f"The client inherited {inheritance_value} {currency} in {inheritance.get('inheritance year')} "
        f"from their {inheritance.get('relationship')}, who was a {inheritance.get('profession')}."
        if inheritance else "No inheritance details provided."
    )
    
    real_estate_list = client_profile['client_profile']["real_estate_details"]
    if real_estate_list:
        real_estate_texts = [
            f"a {prop['property type']} in {prop['property location']} valued at {prop['property value']} {currency}"
            for prop in real_estate_list
        ]
        real_estate_text = "The client owns " + ", and ".join(real_estate_texts) + "."
    else:
        real_estate_text = "No real estate owned."
    savings_text = f"The client has savings of {savings} {currency} while working."


    # Final prompt text
    prompt_text = f"{savings_text} {real_estate_text} {inheritance_text}"
    return prompt_text

print(generate_wealth_summary_text(full_data[3]))

The client has savings of 1450000 EUR while working. The client owns a condo in Livorno valued at 995000 EUR, and a villa in Udine valued at 1830000 EUR. The client inherited 2690000 EUR in 2013 from their mother, who was a Corporate Lawyer.


In [133]:
def check_wealth_background(client, model):
    # Convert structured wealth data to natural language
    wealth_summary = generate_wealth_summary_text(client)

    # Construct the input text for the model
    input_text = (
        "Wealth History: " + wealth_summary + " " +
        "Wealth Description: " + str(client["client_description"].get("Wealth Summary", ""))
    )

    # Call the LLM to validate consistency
    response = ollama.chat(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a logical AI assistant. Based on the user's input, check if the structured wealth History "
                    "is coherent with the client's wealth history description. Only reply with 'True' if they are consistent, otherwise reply with 'False'."
                )
            },
            {"role": "user", "content": input_text}
        ],
        options={"num_predict": 5, "temperature": 0.0}
    )

    return response['message']['content']


In [131]:
for i in full_data:
    if i['account_form']['currency']=="EUR" or i['account_form']['currency']=="CHF" or i['account_form']['currency']=="DKK":
        continue
    else:
        print(i['account_form']['currency'])


In [144]:
check_wealth_background(full_data[500], 'gemma3')

'True'

In [157]:
full_data[600]['client_description']['Client Summary'] #full_data[200]['client_description']['Client Summary']

'The RM knows the client very well and is a good friend of the entire family.\n'

In [158]:
def check_client_summary(client, model):
    # Extract the client summary text
    summary_text = client['client_description']['Client Summary']
    input_text = "Client Summary: " + summary_text

    response = ollama.chat(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a logical AI assistant. Based on the user's input, determine whether the client summary contains "
                    "a positive recommendation. Only reply with 'True' if it clearly recommends favors the client, otherwise reply with 'False'."
                )
            },
            {"role": "user", "content": input_text}
        ],
        options={"num_predict": 5, "temperature": 0.0}
    )

    return response['message']['content']


In [159]:
check_client_summary(full_data[600], 'gemma3')

'False\n'

In [7]:
def parse_boolean_answers(answer_list):
    parsed_flags = []
    for answer in answer_list:
        # Normalize and strip whitespace/newlines
        answer_clean = str(answer).strip().lower()
        
        # Check if the cleaned answer starts with 'false' (can catch 'false\n', 'false.', etc.)
        if 'false' in answer_clean:
            parsed_flags.append(False)
        elif 'true' in answer_clean:
            parsed_flags.append(True)
        else:
            raise ValueError(f"Unexpected answer format: {answer_clean}")
    
    return parsed_flags


In [ ]:
family_background = []
education_background = []
work_background = []
wealth_background = []
client_summ = []
for client in full_data:
    family_background.append(check_family_background(client,'gemma3'))
    education_background.append(check_education_background(client,'gemma3'))
    work_background.append(check_work_background(client,'gemma3'))
    wealth_background.append(check_wealth_background(client,'gemma3'))
    client_summ.append(check_client_summary(client,'gemma3'))
    

In [ ]:
from concurrent.futures import ThreadPoolExecutor

model = 'gemma3'
family_background = []
education_background = []
work_background = []
wealth_background = []
client_summ = []

def run_all_checks(client):
    return (
        check_family_background(client, model),
        check_education_background(client, model),
        check_work_background(client, model),
        check_wealth_background(client, model),
        check_client_summary(client, model)
    )

with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(run_all_checks, full_data))

for fam, edu, work, wealth, summ in results:
    family_background.append(fam)
    education_background.append(edu)
    work_background.append(work)
    wealth_background.append(wealth)
    client_summ.append(summ)

In [ ]:
family_background_parsed = parse_boolean_answers(family_background)
education_background_parsed = parse_boolean_answers(education_background)
work_background_parsed = parse_boolean_answers(work_background)
wealth_background_parsed = parse_boolean_answers(wealth_background)
client_summ_parsed = parse_boolean_answers(client_summ)


In [ ]:
import time as t
start = t.time()
i = 0
family_background = []
for client in full_data:
    i= i+1
    family_background.append(check_family_background(client,'llama3.2'))
    if i%100==0:
        print(i)
        end = t.time()
        print("Time taken for family background check:", end-start)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [8]:
family_background_parsed = parse_boolean_answers(family_background)

In [9]:
from sklearn.metrics import confusion_matrix
import numpy as np
confusion_matrix(np.array(family_background_parsed), np.array(client_labels))

array([[ 296,   86],
       [1489, 4922]], dtype=int64)